In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import matplotlib.colors as clr

In [ ]:
"""
Recebe uma imagem e altera as suas dimensões para (m,n) multiplas de (16,16).
Isto é realizado através da cópia da ultima coluna/linha até atingir o valor multiplo de 16.
"""
def padding(img : np.array):
    img = img.copy()
    while img.shape[0]%16 != 0:
        img = np.concatenate((img, [img[-1]]), axis = 0)
    while img.shape[1]%16 != 0:
        img = np.concatenate((img, [img[:,-1]]), axis = 1)
    return img

"""
Recebe uma imagem e, enquanto a ultima coluna/imagem for igual à penultima, elimina-a
"""
def unpad(img : np.array):
    img = img.copy()
    while np.all(img[-1] == img[-2]):
        if len(img.shape) == 2:
            img = np.resize(img, (img.shape[0] - 1, img.shape[1]) )
        else:
            img = np.resize(img, (img.shape[0] - 1, img.shape[1], img.shape[2] ) )
    while np.all(img[:, -1] == img[:, -2]):
        if len(img.shape) == 2:
            img = np.resize(img, (img.shape[0], img.shape[1] - 1) )
        else:
            img = np.resize(img, (img.shape[0], img.shape[1] - 1, img.shape[2] ) )
    return img

In [ ]:
p_img3 = unpad(p_img2)

In [ ]:
p_img = im.imread('imagens/Low/barn_mountains.jpg')
p_img2 = padding(p_img)
p_img3 = unpad(p_img2)

In [ ]:
np.all(np.equal(p_img3, p_img))

In [ ]:
img.shape

In [ ]:
np.hstack(p_img).shape

In [ ]:
p_img.shape

In [ ]:
"""
Converte imagem no formato RGB para imagem no formato yCbCr;
"""
def rgb_to_ycbcr(img : np.array):
    img.copy()
    y = 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114*img[:,:,2]
    cb = (img[:,:,2] - y)/1.772 + 128
    cr = (img[:,:,0] - y)/1.402 + 128
    
    y_cb_cr = np.dstack((y, cb, cr))
    return y_cb_cr
"""
Converte imagem no formato yCbCr para imagem no formato RGB;
"""
def ycbcr_to_rgb(img : np.array):
    img = img.copy()
    y = img[:,:,0]
    cb = img[:, :, 1] - 128
    cr = img[:, :, 2] - 128
    
    r = y + 1.402 * cr
    g = y - 0.344136*cb - 0.714136*cr
    b = y + 1.772*cb
    
    rgb = np.dstack((r,g,b))
    np.round(rgb)
    rgb[rgb>255] = 255
    rgb[rgb < 0] = 0
    
    return np.array(rgb, dtype = np.uint8)

In [ ]:
ycbr = rgb_to_ycbcr(p_img2)